In [1]:
from _plotly_future_ import v4_subplots

In [2]:
from scipy import stats

In [3]:
import pandas as pd
import numpy as np

In [4]:
from dgutils.pandas import add_column, add_columns

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
from collections import Counter

In [6]:
from utils import EvalMetric

Using TensorFlow backend.
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/tensorflow/python/framework/tensor_util.py:32: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  from tensorflow.python.framework import fast_tensor_util
/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  from . import _ni_label


In [7]:
import matplotlib.pyplot as plt 
# import seaborn as sns
import pandas as pd
# sns.set(color_codes=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(theme='ggplot')

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/cluster/vq.py:77: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/cluster/hierarchy.py:178: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff
init_notebook_mode(connected=True)

In [9]:
result_filenmae = 'result/prediction.rfam.rnafold_100.2019_10_18_1_ep10.sample100.pkl'

In [10]:
df = pd.read_pickle(result_filenmae)


In [13]:
# # tmp load rnafold & clean data TODO remove once rerun prediction
# df_tmp = pd.read_pickle('result/rfam_rnafold_sample_100.pkl.gz')
# df_tmp = df_tmp[['seq_id', 'rnafold_one_idx']]
# df = pd.merge(df, df_tmp, on='seq_id', how='inner')
df = df.rename(columns={'one_idx': 'truth_one_idx', 'rnafold_one_idx': 'one_idx'})
# # tmp fix
# df = add_column(df, 'pred_idx', ['pred_idx'], lambda l: [item for sublist in l for item in sublist])

In [14]:
df.head()

,len,pred_idx,one_idx,seq,seq_id,truth_one_idx
0,117,"[([13, 15, 16, 17, 18, 30, 31, 76, 77, 78, 79,...","[([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15,...",UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,RF00001_A,"([0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 16, 17, 1..."
1,150,"[([6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19,...","[([3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 17, 18, 1...",UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...,RF00002_B,"([32, 33, 35, 36, 43, 44, 45, 46, 47, 58, 59, ..."
2,161,"[([13, 14, 35, 36, 37, 38, 39, 40, 41, 42, 44,...","[([2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16...",AUACUUACCUGGACGGGGUCAAUGGAUGAUCUAUAAGGUCCAUGGC...,RF00003_B,"([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, ..."
3,145,"[([6, 7, 8, 10, 11, 12, 13, 26, 27, 28, 29, 30...","[([2, 3, 4, 5, 6, 10, 11, 12, 13, 27, 28, 29, ...",AUAUCUUCUCGGCUAUUUAGCUAAGAUCAAGUUAUUAAACUGUUCU...,RF00004_A,"([6, 7, 8, 10, 11, 12, 13, 46, 47, 48, 49, 50,..."
4,72,"[([0, 1, 2, 3, 4, 5, 6, 47, 48, 49, 50, 51], [...","[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1...",GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...,RF00005_A,"([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ..."


In [15]:
df[['len']].iplot(kind='histogram', bins=100)

In [16]:
# convert to tuple so we can use set()
def _to_tuple(idxes):
    x = []
    for _a in idxes:
        a = []
        for i, j in zip(_a[0], _a[1]):
            a.append((i, j))
        if len(a) > 0:
            a = tuple(a)
            x.append(a)
    return x

In [17]:
def _to_tuple_single(idx):
    a = []
    for i, j in zip(idx[0], idx[1]):
        a.append((i, j))
    if len(a) > 0:
        a = tuple(a)
        return a
    else:
        return None

In [18]:
def _to_mat(x, l):
    # x is tuple of tuple (2 elements)
    # convert to list of 2 lists
    y = [[a[0] for a in x], [a[1] for a in x]]
    z = np.zeros((l, l))
    z[y] = 1
    return z

In [19]:
n_no_struct = 0
df_metric = []

for _, row in df.iterrows():
    seq = row['seq']
    rnafold_struct_count = Counter(_to_tuple(row['one_idx']))
    model_struct_count = Counter(_to_tuple(row['pred_idx']))
#     # only keep structures that occurs more than 2 times (2% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=2}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=2}
#     # only keep structures that occurs more than 5 times (5% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=5}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=5}
#     # only keep structures that occurs more than 10 times (10% in sampled structures)
#     rnafold_struct_count = {k: v for k, v in rnafold_struct_count.iteritems() if v >=10}
#     model_struct_count = {k: v for k, v in model_struct_count.iteritems() if v >=10}
    
    # skip the case if RNAfold outputs no structure
    if len(rnafold_struct_count) == 0:
        n_no_struct += 1
        continue
    
    # for each structure from RNAfold
    # evaluate against all structures from the model, pick the closest one, record performance
#     result = []
    for rnafold_struct, rnafold_count in rnafold_struct_count.iteritems():
        r_s = _to_mat(rnafold_struct, len(seq))
        sensitivity = 0
        ppv = 0
        f_measure = 0
        best_m_s = None
        for model_struct in model_struct_count.keys():
            m_s = _to_mat(model_struct, len(seq))
            _sensitivity = EvalMetric.sensitivity(m_s, r_s)
            _ppv = EvalMetric.ppv(m_s, r_s)
            _f_measure = EvalMetric.f_measure(_sensitivity, _ppv)
            if _f_measure > f_measure:
                sensitivity = _sensitivity + 0  # copy
                ppv = _ppv + 0
                f_measure = _f_measure + 0
                best_m_s = list(model_struct)
#         result.append((sensitivity, ppv, f_measure))
        df_metric.append({
            'seq': seq,
            'rnafold_idx': list(rnafold_struct),
            'closest_model_idx': best_m_s,
            'sensitivity': sensitivity,
            'ppv': ppv,
            'f_measure': f_measure,
            'rnafold_count': rnafold_count,
        })
    
#     print rnafold_struct_count
#     print model_struct_count
#     print result
#     print ''
    
    
print n_no_struct
df_metric = pd.DataFrame(df_metric)


/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

utils.py:325: RuntimeWarning:

invalid value encountered in double_scalars



0


In [20]:
# TODO add seq_id in above code
df_metric = pd.merge(df_metric, df[['seq', 'len']], on='seq', how='left')

In [21]:
df_metric.head()

,closest_model_idx,f_measure,ppv,rnafold_count,rnafold_idx,sensitivity,seq,len
0,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.876712,0.914286,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.842105,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
1,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.849315,0.885714,1,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.815789,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
2,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.914286,0.941176,2,"[(2, 113), (3, 112), (4, 111), (5, 110), (6, 1...",0.888889,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
3,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.853333,0.914286,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.800000,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117
4,"[(0, 115), (1, 114), (2, 113), (3, 112), (4, 1...",0.901408,0.914286,1,"[(1, 114), (2, 113), (3, 112), (4, 111), (5, 1...",0.888889,UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,117


In [22]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Sensitivity', 'PPV'])

for col_idx, metric_name in enumerate(['sensitivity', 'ppv']):

    col = col_idx + 1
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='1 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 1])),
                              marker_color='#ffffcc', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='2 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 2])),
                              marker_color='#c7e9b4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='3 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 3])),
                              marker_color='#7fcdbb', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='4 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 4])),
                              marker_color='#41b6c4', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='5 ({})'.format(len(df_metric[df_metric['rnafold_count'] == 5])),
                              marker_color='#1d91c0', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
                              nbinsx=20, histnorm='probability', 
                               name='(5, 10] ({})'.format(len(df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)])),
                              marker_color='#225ea8', showlegend=True if col_idx == 0 else False),
                 row=1, col=col)
    fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
                              nbinsx=20, histnorm='probability', 
                               name='>10 ({})'.format(len(df_metric[df_metric['rnafold_count'] > 10])),
                              marker_color='#0c2c84', showlegend=True if col_idx == 0 else False), 
                 row=1, col=col)

fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, zeroline=False,
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                showline=True, mirror=True),
#     yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                tickfont=dict(size=18, color='#000'),
#                titlefont=dict(size=24, color='#000'),
#                title='Percentage', showline=True, mirror=True),
    title=result_filenmae,
#     title=metric_name,
#     titlefont=dict(size=24, color='#000'),
#     legend=dict(orientation="h",
#                 font=dict(
# #             family='sans-serif',
#             size=24,
#             color='#000'
#         ),),
))
iplot(fig)

In [23]:
# for metric_name in ['sensitivity', 'ppv']:
#     fig = go.Figure()
#     # fig = df_metric[df_metric['rnafold_count'] > 5][['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
#     #                                                     histnorm='probability', opacity=1,
#     #                                                      barmode='group', asFigure=True)

#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 1][metric_name], 
#                               nbinsx=20, histnorm='probability', name='1'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 2][metric_name], 
#                               nbinsx=20, histnorm='probability', name='2'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 3][metric_name], 
#                               nbinsx=20, histnorm='probability', name='3'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 4][metric_name], 
#                               nbinsx=20, histnorm='probability', name='4'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] == 5][metric_name], 
#                               nbinsx=20, histnorm='probability', name='5'))
#     fig.add_trace(go.Histogram(x=df_metric[(df_metric['rnafold_count'] > 5) & (df_metric['rnafold_count'] <= 10)]['sensitivity'], 
#                               nbinsx=20, histnorm='probability', name='(5, 10]'))
#     fig.add_trace(go.Histogram(x=df_metric[df_metric['rnafold_count'] > 10][metric_name], 
#                               nbinsx=20, histnorm='probability', name='>10'))

#     fig = fig.update(layout=dict(
#         paper_bgcolor='rgba(0,0,0,0)',
#         plot_bgcolor='rgba(0,0,0,0)',
#         xaxis=dict(showgrid=False, zeroline=False,
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    showline=True, mirror=True),
#         yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
#                    tickfont=dict(size=18, color='#000'),
#                    titlefont=dict(size=24, color='#000'),
#                    title='Percentage', showline=True, mirror=True),
#     #     title='Performance on test dataset',
#         title=metric_name,
#         titlefont=dict(size=24, color='#000'),
#     #     legend=dict(orientation="h",
#     #                 font=dict(
#     # #             family='sans-serif',
#     #             size=24,
#     #             color='#000'
#     #         ),),
#     ))
#     iplot(fig)

In [24]:
def make_hist_2(df_plot, title):
#     fig = df_plot[['sensitivity', 'ppv']].iplot(kind='histogram', bins=20,
#                                                     histnorm='probability', opacity=1,
#                                                      barmode='group', asFigure=True)
#     fig = df_plot[['sensitivity', 'ppv']].iplot(kind='histogram', xbins=dict(
#                                                                     start=0.0,
#                                                                     end=1.0,
#                                                                     size=0.05
#                                                                 ),
#                                                     histnorm='probability', opacity=1,
#                                                      barmode='group', asFigure=True)
    fig = go.Figure()
    fig.add_trace(go.Histogram(
    x=df_plot['sensitivity'],
    histnorm='percent',
    name='Sensitivity', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        start=0.0,
        end=1.0,
        size=0.05
    ),
    marker_color='#EB89B5',
    opacity=0.75
    ))
    fig.add_trace(go.Histogram(
    x=df_plot['ppv'],
    histnorm='percent',
    name='PPV',
    xbins=dict( # bins used for histogram
        start=0.0,
        end=1.0,
        size=0.05
    ),
    marker_color='#330C73',
    opacity=0.75
    ))

    fig = fig.update(layout=dict(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=False, zeroline=False,
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               showline=True, mirror=True,
              range=[0,1]),
    yaxis=dict(showgrid=True, gridwidth=1, gridcolor='darkgray',
               tickfont=dict(size=18, color='#000'),
               titlefont=dict(size=24, color='#000'),
               title='Percentage', showline=True, mirror=True),
    title=title,
    titlefont=dict(size=24, color='#000'),
    legend=dict(orientation="h",
                font=dict(
            size=24,
            color='#000'
        ),),
    ))
    return fig

In [25]:
fig = make_hist_2(df_metric[(df_metric['rnafold_count']>=5) & (df_metric['len'] <= 100)],
                 'count>=5 & len<=100')
iplot(fig)

In [26]:
fig = make_hist_2(df_metric[(df_metric['rnafold_count']>=5) & (df_metric['len'] > 100)],
                 'count>=5 & len>100')
iplot(fig)

In [28]:
# # uncomment to show plot in new window
# plot(fig)

In [29]:
len(df_metric)

10245

In [30]:
sum(df_metric['ppv']>0.8)/float(len(df_metric))

0.7494387506100537

In [31]:
# # for R plot
# df_metric[['seq', 'sensitivity', 'ppv', 'f_measure']].to_csv('/Users/alicegao/data/tmp/perf_rand_ensemble_metric.csv', index=False)

In [32]:
# # scatter plot

# fig = df_metric[['sensitivity', 'ppv']].iplot(kind='scatter', x='sensitivity', y='ppv',
#                                                     mode='markers', size=4,
#                                               asFigure=True)
# fig = fig.update(layout=dict(
#     paper_bgcolor='rgba(0,0,0,0)',
#     plot_bgcolor='rgba(0,0,0,0)',
#     xaxis=dict(showgrid=False, title='Sensitivity', zeroline=False),
#     yaxis=dict(showgrid=False, title='PPV'),
#     title='Performance on test dataset',
# ))

In [33]:
# iplot(fig)

In [34]:
df.head()

,len,pred_idx,one_idx,seq,seq_id,truth_one_idx
0,117,"[([13, 15, 16, 17, 18, 30, 31, 76, 77, 78, 79,...","[([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15,...",UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,RF00001_A,"([0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 16, 17, 1..."
1,150,"[([6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19,...","[([3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 17, 18, 1...",UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...,RF00002_B,"([32, 33, 35, 36, 43, 44, 45, 46, 47, 58, 59, ..."
2,161,"[([13, 14, 35, 36, 37, 38, 39, 40, 41, 42, 44,...","[([2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16...",AUACUUACCUGGACGGGGUCAAUGGAUGAUCUAUAAGGUCCAUGGC...,RF00003_B,"([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, ..."
3,145,"[([6, 7, 8, 10, 11, 12, 13, 26, 27, 28, 29, 30...","[([2, 3, 4, 5, 6, 10, 11, 12, 13, 27, 28, 29, ...",AUAUCUUCUCGGCUAUUUAGCUAAGAUCAAGUUAUUAAACUGUUCU...,RF00004_A,"([6, 7, 8, 10, 11, 12, 13, 46, 47, 48, 49, 50,..."
4,72,"[([0, 1, 2, 3, 4, 5, 6, 47, 48, 49, 50, 51], [...","[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1...",GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...,RF00005_A,"([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ..."


In [35]:
# check metrix against 'ground truth'
df2 = df.copy()


def _compute_metric_gt(true_idx, pred_idx, len_seq):
    _true_idx = _to_tuple_single(true_idx)
    if _true_idx:
        m_true = _to_mat(_true_idx, len_seq)
    else:
        m_true = np.zeros((len_seq, len_seq))
    s = []
    p = []
    f = []
    for p_idx in _to_tuple(pred_idx):
#         print p_idx
        m_p = _to_mat(p_idx, len_seq)
        _sensitivity = EvalMetric.sensitivity(m_true, m_p)
        _ppv = EvalMetric.ppv(m_true, m_p)
        _f_measure = EvalMetric.f_measure(_sensitivity, _ppv)
        s.append(_sensitivity)
        p.append(_ppv)
        f.append(_f_measure)
    return s, p, f


df2 = add_columns(df2, ['rnafold_sensitivity', 'rnafold_ppv', 'rnafold_f'],
                 ['truth_one_idx', 'one_idx', 'len'], _compute_metric_gt)
df2 = add_columns(df2, ['pred_sensitivity', 'pred_ppv', 'pred_f'],
                 ['truth_one_idx', 'pred_idx', 'len'], _compute_metric_gt)



/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



In [36]:
df2.head()

,len,one_idx,pred_f,pred_idx,pred_ppv,pred_sensitivity,rnafold_f,rnafold_ppv,rnafold_sensitivity,seq,seq_id,truth_one_idx
0,117,"[([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15,...","[0.12500000000000003, 0.3157894736842105, 0.07...","[([13, 15, 16, 17, 18, 30, 31, 76, 77, 78, 79,...","[0.0909090909091, 0.272727272727, 0.0606060606...","[0.2, 0.375, 0.1111111111111111, 0.4, 0.214285...","[0.5070422535211268, 0.6285714285714286, 0.753...","[0.545454545455, 0.666666666667, 0.78787878787...","[0.47368421052631576, 0.5945945945945946, 0.72...",UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,RF00001_A,"([0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 16, 17, 1..."
1,150,"[([3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 17, 18, 1...","[0.32142857142857145, 0.5384615384615384, 0.40...","[([6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19,...","[0.346153846154, 0.538461538462, 0.34615384615...","[0.3, 0.5384615384615384, 0.5, 0.4242424242424...","[0.3733333333333333, 0.3888888888888889, 0.411...","[0.538461538462, 0.538461538462, 0.53846153846...","[0.2857142857142857, 0.30434782608695654, 0.33...",UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...,RF00002_B,"([32, 33, 35, 36, 43, 44, 45, 46, 47, 58, 59, ..."
2,161,"[([2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16...","[0.07692307692307693, 0.2571428571428572, 0.27...","[([13, 14, 35, 36, 37, 38, 39, 40, 41, 42, 44,...","[0.075, 0.225, 0.225, 0.075, 0.075, 0.35, 0.35...","[0.07894736842105263, 0.3, 0.34615384615384615...","[0.2888888888888889, 0.3076923076923077, 0.239...","[0.325, 0.35, 0.275, 0.35, 0.325, 0.325, 0.2, ...","[0.26, 0.27450980392156865, 0.2115384615384615...",AUACUUACCUGGACGGGGUCAAUGGAUGAUCUAUAAGGUCCAUGGC...,RF00003_B,"([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, ..."
3,145,"[([2, 3, 4, 5, 6, 10, 11, 12, 13, 27, 28, 29, ...","[0.6956521739130435, 0.2325581395348837, 0.857...","[([6, 7, 8, 10, 11, 12, 13, 26, 27, 28, 29, 30...","[0.8, 0.166666666667, 0.8, 0.966666666667, 0.7...","[0.6153846153846154, 0.38461538461538464, 0.92...","[0.5753424657534246, 0.8219178082191781, 0.533...","[0.7, 1.0, 0.666666666667, 0.766666666667, 0.8...","[0.4883720930232558, 0.6976744186046512, 0.444...",AUAUCUUCUCGGCUAUUUAGCUAAGAUCAAGUUAUUAAACUGUUCU...,RF00004_A,"([6, 7, 8, 10, 11, 12, 13, 46, 47, 48, 49, 50,..."
4,72,"[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1...","[0.7272727272727273, 0.5789473684210527, 0.8, ...","[([0, 1, 2, 3, 4, 5, 6, 47, 48, 49, 50, 51], [...","[0.571428571429, 0.52380952381, 0.761904761905...","[1.0, 0.6470588235294118, 0.8421052631578947, ...","[0.5, 0.5217391304347826, 0.5333333333333333, ...","[0.571428571429, 0.571428571429, 0.57142857142...","[0.4444444444444444, 0.48, 0.5, 0.48, 0.461538...",GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...,RF00005_A,"([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ..."


In [37]:
# _row = df2.iloc[4]

_idx = np.random.randint(len(df2))
_row = df2.iloc[_idx]
print _idx, _row['seq_id'], _row['len']

# fig = go.Figure()
fig = make_subplots(rows=3, cols=1, subplot_titles=['Sensitivity', 'PPV', 'F'])

fig.add_trace(go.Histogram(
x=_row['rnafold_ppv'],
histnorm='percent',
name='rnafold_ppv', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#0ea7b5',
opacity=0.75
), row=2, col=1)
fig.add_trace(go.Histogram(
x=_row['pred_ppv'],
histnorm='percent',
name='pred_ppv', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#e8702a',
opacity=0.75
), row=2, col=1)

fig.add_trace(go.Histogram(
x=_row['rnafold_sensitivity'],
histnorm='percent',
name='rnafold_sensitivity', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#6bd2db',
opacity=0.75
), row=1, col=1)
fig.add_trace(go.Histogram(
x=_row['pred_sensitivity'],
histnorm='percent',
name='pred_sensitivity', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#ffbe4f',
opacity=0.75
), row=1, col=1)

fig.add_trace(go.Histogram(
x=_row['rnafold_f'],
histnorm='percent',
name='rnafold_f', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#6bd2db',
opacity=0.75
), row=3, col=1)
fig.add_trace(go.Histogram(
x=_row['pred_f'],
histnorm='percent',
name='pred_f', # name used in legend and hover labels
xbins=dict( # bins used for histogram
    start=0.0,
    end=1.0,
    size=0.05
),
marker_color='#ffbe4f',
opacity=0.75
), row=3, col=1)
iplot(fig)

113 RF00433_B 152


In [38]:
n_better = 0
n_worse = 0
n_no_diff = 0
for _, row in df2.iterrows():
    print row['seq_id'], row['len']
    pred_f = np.asarray(row['pred_f'])
    rnafold_f = np.asarray(row['rnafold_f'])
    pred_f = pred_f[~np.isnan(pred_f)]
    rnafold_f = rnafold_f[~np.isnan(rnafold_f)]
    
    # tmp FIXME
    if len(pred_f) == 0:
        print("Skip (all NaNs?)")
        continue
    
    print np.mean(pred_f), np.mean(rnafold_f)
    ks, pval = stats.ks_2samp(pred_f, rnafold_f)
    print ks, pval
    print ''
    
    if pval >= 0.05:
        n_no_diff += 1
    elif np.mean(pred_f) >= np.mean(rnafold_f):
        n_better += 1
    else:
        assert np.mean(pred_f) < np.mean(rnafold_f)
        assert pval < 0.05
        n_worse += 1
    
print "n_better: {}, n_worse: {}, n_no_diff: {}".format(n_better, n_worse, n_no_diff)

RF00001_A 117
0.2957599916818284 0.6291277631206762
0.7852173913043478 4.189751560265265e-27

RF00002_B 150
0.4337096467730072 0.3725877667355188
0.71 3.637519451670771e-23

RF00003_B 161
0.18959413715955487 0.2913847801940778
0.5131313131313131 1.344560132405666e-10

RF00004_A 145
0.6437973364831601 0.6109723183682091
0.26 0.001784667421980056

RF00005_A 72
0.6663573412540177 0.5351353165499626
0.76 1.758654071870821e-26

RF00007_B 129
0.7024745340530311 0.7284825457468455
0.36 2.8490733934581266e-06

RF00008_B 54
0.7557888861200598 0.8931975067039778
0.51 3.696390026378737e-12

RF00009_A 320
0.14063157703205167 0.14506356090262054
0.4536082474226804 1.957578493844417e-09

RF00010_A 312
0.4258304576591742 0.5054829474997481
0.54 1.402153914254165e-13

RF00011_B 374
0.45544810180388795 0.4854600463772749
0.18000000000000005 0.06909243488939809

RF00012_B 219
0.6169763434146709 0.7976983940843259
0.7000000000000001 1.5734899709005905e-22

RF00013_A 183
0.3855741125705699 0.8497452439849

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning:

Mean of empty slice.

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning:

invalid value encountered in double_scalars

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/stats/stats.py:4862: RuntimeWarning:

invalid value encountered in true_divide

/Users/alicegao/anaconda2/envs/dg_work_old_tf/lib/python2.7/site-packages/scipy/stats/stats.py:4867: RuntimeWarning:

divide by zero encountered in double_scalars



AssertionError: 

In [ ]:
# TODO use unique structures only?

In [ ]:
# TODO calculate one statistics per row
# that the two f_measure distribution is different
# use the mean to determine direction
# report number of rows where they are not significantly different, where A > B, where B > A

In [ ]:
# generate 1000 structures

In [ ]:
# most likely
# compare distribution
# test time: most likely path
# every step: run 10 examples of probability matrix, sample 100 from the 10
# then take the top-10 sampled paths, then proceed to next step
